Hyper-parameter search with the Text-To-Text Transformer 🤖 (Bayes)
-----------------------------------

We have already tried to fine-tune the Text-To-Text Transformer on the original sentences. Now that we extracted new sentences from the book intituled **Grammaire de Wolof Moderne**, we want to verify if the T5 model will perform better on them than previously. We will use, again, the `Bayesian Hyperparamter Optimization` to search for the best combination of hyperparameter (the best model). The `wandb` library will be used to make a clear evaluation of the model with the following charts `Parallel coordinate` and `Parameter Importance`. After finding the best model, we will take the checkpoints and continue the training in another notebook. Let us dive into the process.

We want to know the best combination of values of the following hyperparameters:

- **learning rate** $\sim Log U(1e-2, 1e-5)$
- **weight decay** $\in \{0.0, 0.1, 0.2, 0.3, 0.4, 0.5\}$
- **random state** (seed of the data splitting generator) $\in range(1, 100)$ 

1. For the translation from French to Wolof

  - **fr_char_p** (probability of modifying a character from a French word) $\sim U(0.0, 0.9)$
  - **fr_word_p** (probability of modifying a word from a French sentence) $\sim U(0.0, 0.9)$

2. For the translation from Wolof to French

  - **wf_char_p** (probability of modifying a character from a Wolof word) $\sim U(0.0, 0.9)$
  - **fr_word_p** (probability of modifying a word from a Wolof sentence) $\sim U(0.0, 0.9)$


The Bayes method requires to define a metric. We will evaluate the model on the test set, so the metric that we will add in the hyperparameter setting can be either the `cross entropy loss` calculated on the test set or `BLEU` score. Since it is a machine translation, a BLEU score will be more useful as evaluation metric. 

**Objective**: We will try to `maximize the metric.` For the moment, we want to obtain a `BLEU` score more than `20`.

In [1]:
# let us extend the paths of the system
import sys

path = "/content/drive/MyDrive/Memoire/subject2/T5/"

sys.path.extend([path, f"{path}new_data"])

In [2]:
# define wandb environment
%env WANDB_LOG_MODEL=true
%env WANDB_NOTEBOOK_NAME=/content/drive/MyDrive/Memoire/subject2/T5/hp_search_t5_small_step_with_bayes_v2.ipynb
%env WANDB_API_KEY=237a8450cd2568ea1c8e1f8e0400708e79b6b4ee 

env: WANDB_LOG_MODEL=true
env: WANDB_NOTEBOOK_NAME=/content/drive/MyDrive/Memoire/subject2/T5/hp_search_t5_small_step_with_bayes_v2.ipynb
env: WANDB_API_KEY=237a8450cd2568ea1c8e1f8e0400708e79b6b4ee


In [3]:
!pip install -qq wandb --upgrade

In [4]:
!pip install evaluate -qq
!pip install sacrebleu -qq
# !pip install optuna -qq
!pip install transformers -qq 
!pip install tokenizers -qq
!pip install nlpaug -qq
!pip install ray[tune] -qq
!python -m spacy download fr_core_news_lg 

2023-05-06 11:40:21.471489: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-06 11:40:23.485109: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-05-06 11:40:26.774039: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-06 11:40:26.774773: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returnin

In [5]:
# let us import all necessary libraries
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer, T5TokenizerFast, set_seed
from wolof_translate.utils.sent_transformers import TransformerSequences
from wolof_translate.data.dataset_v2 import T5SentenceDataset
from wolof_translate.utils.sent_corrections import *
from sklearn.model_selection import train_test_split
from nlpaug.augmenter import char as nac
from torch.utils.data import DataLoader
# from datasets  import load_metric # make pip install evaluate instead
# and pip install sacrebleu for instance
from functools import partial
from tqdm import tqdm
import pandas as pd
import numpy as np
import evaluate
import wandb
import torch


We will create two models: 

- One translating the french corpus to a wolof corpus [french_to_wolof](#french-to-wolof)
- One translating the wolof corpus to a french corpus [wolof_to_french](#wolof-to-french)

--------------

## French to wolof

### Configure dataset 🔠

We will split the sentences between train (for the model's training), validation (to find the best performance) and test (to make final predictions) sets. The samples added as train, validation and test sets are identified according to `the random state.` We will tune the random state to the groups that guarantee the model's best fitting. In other words, we want the model to identify many training sentences and generalize that learning on the validation sentences. It is not sometimes the case, mainly when using a small dataset like ours. 

Notice that when continuing to train the model from the checkpoints we will use the train_set plus the validation set. Then we need to save the dataset part which doesn't contain the test set for latter. 

In [6]:
def split_data(random_state: int = 50):
  """Split data between train, validation and test sets

  Args:
    random_state (int): the seed of the splitting generator. Defaults to 50
  """
  # load the corpora and split into train and test sets
  corpora = pd.read_csv(f"{path}diagne_sentences/extractions.csv")

  train_set, test_set = train_test_split(corpora, test_size=0.1, random_state=random_state)

  # let us save the final training set when performing

  train_set, valid_set = train_test_split(train_set, test_size=0.1, random_state=random_state)

  train_set.to_csv(f"{path}diagne_sentences/final_train_set.csv", index=False)

  # let us save the sets
  train_set.to_csv(f"{path}diagne_sentences/train_set.csv", index=False)

  valid_set.to_csv(f"{path}diagne_sentences/valid_set.csv", index=False)

  test_set.to_csv(f"{path}diagne_sentences/test_set.csv", index=False)

Let us load the French and Wolof corpora's common tokenizer.

In [7]:
# recuperate the tokenizer from a json file
tokenizer = T5TokenizerFast(tokenizer_file=f"{path}wolof_translate/tokenizers/t5_tokenizers/tokenizer_v2.json")


The following function will make recuperate the datasets.

In [8]:
def recuperate_datasets(fr_char_p: float, fr_word_p: float):

  # Create augmentation to add on French sentences
  fr_augmentation = TransformerSequences(nac.KeyboardAug(aug_char_p=fr_char_p, aug_word_p=fr_word_p),
                                        remove_mark_space, delete_guillemet_space)

  # Recuperate the train dataset
  train_dataset_aug = T5SentenceDataset(f"{path}diagne_sentences/train_set.csv",
                                        tokenizer,
                                        truncation = True,
                                        cp1_transformer = fr_augmentation)

  # Recuperate the validation dataset
  valid_dataset = T5SentenceDataset(f"{path}diagne_sentences/valid_set.csv",
                                        tokenizer,
                                        truncation = True)
  
  # Return the datasets
  return train_dataset_aug, valid_dataset

### Configure hyperparameter search ⚙️

We have to configure the search space, the search method and the metric. 

In [9]:
wandb.login(key="237a8450cd2568ea1c8e1f8e0400708e79b6b4ee")

# hyperparameters
sweep_config = {
    'method': 'bayes',
    'metric':{
          'goal': 'maximize',
          'name': 'bleu'
      },
    'parameters':
    {
      'epochs': {
          'value': 1
      },
      'batch_size': {
          'values': [16, 32] # we found in previous evaluations that the bleu is better when batch size is not small
      },
      'learning_rate': {
          'distribution': 'log_uniform_values',
          'min': 1e-4,
          'max': 1e-1 # after some evaluations we found out that a high learning rate is better
      },
      'weight_decay': {
          'values': [0.0, 0.1, 0.2, 0.3, 0.4]
      },
     'fr_char_p': {
         'min': 0.2,
         'max': 1.0 
     },
     'fr_word_p': {
          'min': 0.2,
          'max': 0.9
     },
     'random_state': {
         'values': list(range(0, 80))
     }
    }
}

# Initialize the hyperparameter search
sweep_id = wandb.sweep(sweep_config, project = "small-t5-fw-translation-bayes-hpsearch-v2")



wandb: Currently logged in as: oumar-kane (oumar-kane-team). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: tdykh4ig
Sweep URL: https://wandb.ai/oumar-kane-team/base-t5-fw-translation-bayes-hpsearch-v2/sweeps/tdykh4ig


### Configure the model and the evaluation function ⚙️

Let us recuperate the model and resize the token embeddings.

**Note**: In the first training we want to use the t5-small. If we don't obtain good results we will take the t5-base which contains more parameters. See bellow the configuration of the t5-small and the t5-base models, respectively.

In [10]:
small_model_name = 't5-small'
base_model_name = 't5-base'

# import the small model with its pre-trained weights
small_model = AutoModelForSeq2SeqLM.from_pretrained(small_model_name)

# import the base model with its pre-trained weights
base_model = AutoModelForSeq2SeqLM.from_pretrained(base_model_name)


In [11]:
# print the small configuration
small_model.config

T5Config {
  "_name_or_path": "t5-small",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 6,
  "num_heads": 8,
  "num_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_beams": 4,
      "prefix": "summarize: "
    },
    "translation_en_to_de": {
      "early_stopping": true,
      "max_length": 300,
      "num_beams": 4,
      "prefi

In [12]:
# print the base configuration
base_model.config

T5Config {
  "_name_or_path": "t5-base",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_beams": 4,
      "prefix": "summarize: "
    },
    "translation_en_to_de": {
      "early_stopping": true,
      "max_length": 300,
      "num_beams": 4,
      "pre

The small model have the same architecture than the original transformer of Ashish Vaswani, Noam Shazeer, and all. in the article [Attention_is_all_you_need](https://arxiv.org/pdf/1706.03762).

The base model contains more parameters since it use 12 heads in place of 8 and, 12 stack decoder layers in place of 6, the number of feed forward features is of 3072 so 1024 more features than the small one and the embedding dimension is of 768 in place of 512. The base model contains exactly 220 millions of parameters which is a huge number. But since it is pre-trained, we can directly make transfer learning with already trained weights. The base model was firstly explained in the article [Text_To_Text_Transformer](https://arxiv.org/pdf/1910.10683). 

In [13]:
def t5_model_init(tokenizer):

  # Initialize the model name
  model_name = 't5-small'

  # import the model with its pre-trained weights
  model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

  # resize the token embeddings
  model.resize_token_embeddings(len(tokenizer))

  return model

Let us evaluate the predictions with the `bleu` metric. We use a method that we found int the following `HuggingFace` tutorial [translation](https://huggingface.co/docs/transformers/tasks/translation). We will use a class to add more methods if necessary.

In [14]:
# %%writefile wolof-translate/wolof_translate/utils/evaluation.py
from tokenizers import Tokenizer
from typing import *
import numpy as np
import evaluate

class TranslationEvaluation:
    
    def __init__(self, 
                 tokenizer: Tokenizer,
                 decoder: Union[Callable, None] = None,
                 metric = evaluate.load('sacrebleu'),
                 ):
        
        self.tokenizer = tokenizer
        
        self.decoder = decoder
        
        self.metric = metric
    
    def postprocess_text(self, preds, labels):
        
        preds = [pred.strip() for pred in preds]
        
        labels = [[label.strip()] for label in labels]
        
        return preds, labels

    def compute_metrics(self, eval_preds):

        preds, labels = eval_preds

        if isinstance(preds, tuple):
        
            preds = preds[0]
        
        decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

        labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
        
        decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

        decoded_preds, decoded_labels = self.postprocess_text(decoded_preds, decoded_labels)

        result = self.metric.compute(predictions=decoded_preds, references=decoded_labels)
        
        result = {"bleu": result["score"]}

        prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
        
        result["gen_len"] = np.mean(prediction_lens)
        
        result = {k: round(v, 4) for k, v in result.items()}
        
        return result

In [15]:
# %run wolof-translate/wolof_translate/utils/evaluation.py

Let us initialize the evaluation object.

In [16]:
evaluation = TranslationEvaluation(tokenizer)

### Searching for the best parameters 🕖

Let us define the data collator.

In [17]:
def data_collator(batch):
    """Generate a batch of data to provide to trainer

    Args:
        batch (_type_): The batch

    Returns:
        dict: A dictionary containing the ids, the attention mask and the labels
    """
    input_ids = torch.stack([b[0].squeeze(0) for b in batch])
    
    attention_mask = torch.stack([b[1].squeeze(0) for b in batch])
    
    labels = torch.stack([b[2].squeeze(0) for b in batch])
    
    return {'input_ids': input_ids, 'attention_mask': attention_mask,
            'labels': labels}

Let us initialize the training arguments and search for the best model. The latter will be saved as an artefact inside our `wandb` project.

In [ ]:
# %%wandb

def train(config = None):

  with wandb.init(config = config):

    # seed
    set_seed(0)

    # set sweep configuration
    config = wandb.config

    # split the data
    split_data(config.random_state)

    # let us recuperate the datasets
    train_dataset, valid_dataset = recuperate_datasets(config.fr_char_p, config.fr_word_p)

    # set training arguments
    training_args = Seq2SeqTrainingArguments(f"{path}/training/bayes_search_results_fw_v2",
                                      report_to = f"wandb",
                                      num_train_epochs=config.epochs,
                                      load_best_model_at_end=True,
                                      save_strategy="epoch",
                                      evaluation_strategy="epoch",
                                      logging_strategy = 'epoch',
                                      per_device_train_batch_size=config.batch_size, 
                                      per_device_eval_batch_size=16,
                                      learning_rate=config.learning_rate,
                                      weight_decay=config.weight_decay,
                                      predict_with_generate=True, # we will use predict with generate in order to obtain more valuable test results
                                      fp16 = True,
                                      )   

    # define training loop
    trainer = Seq2SeqTrainer(model_init=partial(t5_model_init, tokenizer = train_dataset.tokenizer),
                      args=training_args,
                      train_dataset=train_dataset, 
                      eval_dataset=valid_dataset,
                      data_collator=data_collator,
                      compute_metrics=evaluation.compute_metrics
                      )

    # start training loop
    trainer.train()

agent = wandb.agent(sweep_id, train, count = 100)


wandb: Agent Starting Run: otga3gac with config:
wandb: 	batch_size: 16
wandb: 	epochs: 1
wandb: 	fr_char_p: 0.4381575828545605
wandb: 	fr_word_p: 0.6116586258851745
wandb: 	learning_rate: 0.06259045084863178
wandb: 	random_state: 51
wandb: 	weight_decay: 0.4


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
wandb: WARNING Config item 'learning_rate' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'weight_decay' was locked by 'sweep' (ignored update).


Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,2.843200,nan,0.000000,1.000000


eval/bleu,▁
eval/gen_len,▁
eval/runtime,▁
eval/samples_per_second,▁
eval/steps_per_second,▁
train/epoch,▁▁▁
train/global_step,▁▁▁
train/learning_rate,▁
train/loss,▁
train/total_flos,▁
train/train_loss,▁


wandb: Agent Starting Run: tml8jezj with config:
wandb: 	batch_size: 16
wandb: 	epochs: 1
wandb: 	fr_char_p: 0.8822536140201862
wandb: 	fr_word_p: 0.3339541228740005
wandb: 	learning_rate: 0.00518142881161851
wandb: 	random_state: 72
wandb: 	weight_decay: 0.2


...

eval/bleu,▁
eval/gen_len,▁
eval/loss,▁
eval/runtime,▁
eval/samples_per_second,▁
eval/steps_per_second,▁
train/epoch,▁▁▁
train/global_step,▁▁▁
train/learning_rate,▁
train/loss,▁
train/total_flos,▁


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: a2zcv7z5 with config:
wandb: 	batch_size: 32
wandb: 	epochs: 1
wandb: 	fr_char_p: 0.3987197979448549
wandb: 	fr_word_p: 0.3303424776522219
wandb: 	learning_rate: 0.00215461106235626
wandb: 	random_state: 4
wandb: 	weight_decay: 0


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
wandb: WARNING Config item 'learning_rate' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'weight_decay' was locked by 'sweep' (ignored update).


Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,0.791100,0.560144,0.000000,1.026200


eval/bleu,▁
eval/gen_len,▁
eval/loss,▁
eval/runtime,▁
eval/samples_per_second,▁
eval/steps_per_second,▁
train/epoch,▁▁▁
train/global_step,▁▁▁
train/learning_rate,▁
train/loss,▁
train/total_flos,▁


------------------

## Wolof to french

The only thing that we will change is that is this time it is the Wolof sentences that we will provide as inputs and the French sentences, as targets. 

### Configure dataset 🔠

In [ ]:
def split_data(random_state: int = 50):
  """Split data between train, validation and test sets

  Args:
    random_state (int): the seed of the splitting generator. Defaults to 50
  """
  # load the corpora and split into train and test sets
  corpora = pd.read_csv(f"{path}diagne_sentences/extractions.csv")

  train_set, test_set = train_test_split(corpora, test_size=0.1, random_state=random_state)

  # let us save the final training set when performing

  train_set, valid_set = train_test_split(train_set, test_size=0.1, random_state=random_state)

  train_set.to_csv(f"{path}diagne_sentences/final_train_set.csv", index=False)

  # let us save the sets
  train_set.to_csv(f"{path}diagne_sentences/train_set.csv", index=False)

  valid_set.to_csv(f"{path}diagne_sentences/valid_set.csv", index=False)

  test_set.to_csv(f"{path}diagne_sentences/test_set.csv", index=False)

In [ ]:
# recuperate the tokenizer from a json file
tokenizer = T5TokenizerFast(tokenizer_file=f"{path}wolof_translate/tokenizers/t5_tokenizers/tokenizer_v2.json")

In [ ]:
def recuperate_datasets(wf_char_p: float, wf_word_p: float):

  # Create augmentation to add on Wolof sentences
  wf_augmentation = TransformerSequences(nac.KeyboardAug(aug_char_p=wf_char_p, aug_word_p=wf_word_p),
                                        remove_mark_space, delete_guillemet_space)

  # Recuperate the train dataset
  train_dataset_aug = T5SentenceDataset(f"{path}diagne_sentences/train_set.csv",
                                        tokenizer,
                                        corpus_1 = 'wolof',
                                        corpus_2 = 'french',
                                        truncation = True,
                                        cp1_transformer = wf_augmentation)

  # Recuperate the validation dataset
  valid_dataset = T5SentenceDataset(f"{path}diagne_sentences/valid_set.csv",
                                        tokenizer,
                                        corpus_1 = 'wolof',
                                        corpus_2 = 'french',
                                        truncation = True)
  
  # Return the datasets
  return train_dataset_aug, valid_dataset

### Configure hyperparameter search ⚙️

We have to configure the search space and the search method ("random" in our case). .

In [ ]:
wandb.login(key="237a8450cd2568ea1c8e1f8e0400708e79b6b4ee")

# hyperparameters
sweep_config = {
    'method': 'bayes',
    'metric':{
          'goal': 'maximize',
          'name': 'bleu'
      },
    'parameters':
    {
      'epochs': {
          'value': 1
      },
      'batch_size': {
          'values': [16, 32] 
      },
      'learning_rate': {
          'distribution': 'log_uniform_values',
          'min': 1e-6,
          'max': 1e-2 
      },
      'weight_decay': {
          'values': [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7] 
      },
     'fr_char_p': {
         'min': 0.0,
         'max': 1.0 
     },
     'fr_word_p': {
          'min': 0.0,
          'max': 1.0 
     },
     'random_state': {
         'values': list(range(1, 80))
     }
    }
}

# Initialize the hyperparameter search
sweep_id = wandb.sweep(sweep_config, project = "small-t5-fw-translation-bayes-hpsearch-v2")



wandb: WARNING WANDB_NOTEBOOK_NAME should be a path to a notebook file, couldn't find training_gpt2_2.ipynb.
wandb: Currently logged in as: oumar-kane (oumar-kane-team). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: alygo14y
Sweep URL: https://wandb.ai/oumar-kane-team/gpt2-wolof-french-translation_bayes1_1/sweeps/alygo14y


### Configure the model and the evaluation function ⚙️

Let us recuperate the model and resize the token embeddings.

In [ ]:
def t5_model_init(tokenizer):

  # Initialize the model name
  model_name = 't5-small'

  # import the model with its pre-trained weights
  model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

  # resize the token embeddings
  model.resize_token_embeddings(len(tokenizer))

  return model

Let us evaluate the predictions with the `bleu` metric.

In [ ]:
# %%writefile wolof-translate/wolof_translate/utils/evaluation.py
from tokenizers import Tokenizer
from typing import *
import numpy as np
import evaluate

class TranslationEvaluation:
    
    def __init__(self, 
                 tokenizer: Tokenizer,
                 decoder: Union[Callable, None] = None,
                 metric = evaluate.load('sacrebleu'),
                 ):
        
        self.tokenizer = tokenizer
        
        self.decoder = decoder
        
        self.metric = metric
    
    def postprocess_text(self, preds, labels):
        
        preds = [pred.strip() for pred in preds]
        
        labels = [[label.strip()] for label in labels]
        
        return preds, labels

    def compute_metrics(self, eval_preds):

        preds, labels = eval_preds

        if isinstance(preds, tuple):
        
            preds = preds[0]
        
        decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

        labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
        
        decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

        decoded_preds, decoded_labels = self.postprocess_text(decoded_preds, decoded_labels)

        result = self.metric.compute(predictions=decoded_preds, references=decoded_labels)
        
        result = {"bleu": result["score"]}

        prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
        
        result["gen_len"] = np.mean(prediction_lens)
        
        result = {k: round(v, 4) for k, v in result.items()}
        
        return result

In [ ]:
# %run wolof-translate/wolof_translate/utils/evaluation.py

Let us initialize the evaluation object.

In [ ]:
evaluation = TranslationEvaluation(tokenizer)

### Searching for the best parameters 🕖

Let us define the data collator.

In [ ]:
def data_collator(batch):
    """Generate a batch of data to provide to trainer

    Args:
        batch (_type_): The batch

    Returns:
        dict: A dictionary containing the ids, the attention mask and the labels
    """
    input_ids = torch.stack([b[0].squeeze(0) for b in batch])
    
    attention_mask = torch.stack([b[1].squeeze(0) for b in batch])
    
    labels = torch.stack([b[2].squeeze(0) for b in batch])
    
    return {'input_ids': input_ids, 'attention_mask': attention_mask,
            'labels': labels}

Let us initialize the training arguments and make random search.

In [ ]:
# %%wandb

def train(config = None):

  with wandb.init(config = config):

    # seed
    set_seed(0)

    # set sweep configuration
    config = wandb.config

    # split the data
    split_data(config.random_state)

    # let us recuperate the datasets
    train_dataset, valid_dataset = recuperate_datasets(config.fr_char_p, config.fr_word_p)

    # set training arguments
    training_args = Seq2SeqTrainingArguments(f"{path}/training/bayes_search_results_wf_v2",
                                      report_to = f"wandb",
                                      num_train_epochs=config.epochs,
                                      load_best_model_at_end=True,
                                      save_strategy="epoch",
                                      evaluation_strategy="epoch",
                                      logging_strategy = 'epoch',
                                      per_device_train_batch_size=config.batch_size, 
                                      per_device_eval_batch_size=16,
                                      learning_rate=config.learning_rate,
                                      weight_decay=config.weight_decay,
                                      predict_with_generate=True, # we will use predict with generate in order to obtain more valuable test results
                                      fp16 = True,
                                      )   

    # define training loop
    trainer = Seq2SeqTrainer(model_init=partial(t5_model_init, tokenizer = train_dataset.tokenizer),
                      args=training_args,
                      train_dataset=train_dataset, 
                      eval_dataset=valid_dataset,
                      data_collator=data_collator,
                      compute_metrics=evaluation.compute_metrics
                      )

    # start training loop
    trainer.train()

agent = wandb.agent(sweep_id, train, count = 30)


wandb: Agent Starting Run: a0u0t6k2 with config:
wandb: 	batch_size: 2
wandb: 	epochs: 1
wandb: 	learning_rate: 6.702369179262155e-05
wandb: 	random_state: 30
wandb: 	weight_decay: 0.4
wandb: 	wf_char_p: 0.2819068695463206
wandb: 	wf_word_p: 0.3271474379445852
wandb: WARNING WANDB_NOTEBOOK_NAME should be a path to a notebook file, couldn't find training_gpt2_2.ipynb.


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
wandb: WARNING Config item 'learning_rate' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'weight_decay' was locked by 'sweep' (ignored update).


Epoch,Training Loss,Validation Loss
1,1.331400,0.972646


eval/loss,▁
eval/runtime,▁
eval/samples_per_second,▁
eval/steps_per_second,▁
train/epoch,▁▁▁
train/global_step,▁▁▁
train/learning_rate,▁
train/loss,▁
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: 8p97mqyj with config:
wandb: 	batch_size: 2
wandb: 	epochs: 1
wandb: 	learning_rate: 0.000687378518112751
wandb: 	random_state: 20
wandb: 	weight_decay: 0.2
wandb: 	wf_char_p: 0.10242928904824668
wandb: 	wf_word_p: 0.3761238934195836
wandb: WARNING WANDB_NOTEBOOK_NAME should be a path to a notebook file, couldn't find training_gpt2_2.ipynb.


...

eval/loss,▁
eval/runtime,▁
eval/samples_per_second,▁
eval/steps_per_second,▁
train/epoch,▁▁▁
train/global_step,▁▁▁
train/learning_rate,▁
train/loss,▁
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁


wandb: Agent Starting Run: i305ffth with config:
wandb: 	batch_size: 3
wandb: 	epochs: 1
wandb: 	learning_rate: 4.284698923411304e-05
wandb: 	random_state: 30
wandb: 	weight_decay: 0.4
wandb: 	wf_char_p: 0.6295786463189464
wandb: 	wf_word_p: 0.6968639258681786
wandb: WARNING WANDB_NOTEBOOK_NAME should be a path to a notebook file, couldn't find training_gpt2_2.ipynb.


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
wandb: WARNING Config item 'learning_rate' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'weight_decay' was locked by 'sweep' (ignored update).


Epoch,Training Loss,Validation Loss
1,1.509300,0.986804


eval/loss,▁
eval/runtime,▁
eval/samples_per_second,▁
eval/steps_per_second,▁
train/epoch,▁▁▁
train/global_step,▁▁▁
train/learning_rate,▁
train/loss,▁
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁


-----------

## Colab download and remove step

In [19]:
import shutil

# shutil.rmtree('/content/drive/MyDrive/Memoire/subject2/T5/training/bayes_search_results')
shutil.rmtree('wandb')
# shutil.make_archive('wandb', 'zip', 'wanbd')